##Installation

In [2]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 5.3 MB/s 
     |████████████████████████████████| 636 kB 50.3 MB/s 
     |████████████████████████████████| 52 kB 1.6 MB/s 
     |████████████████████████████████| 3.3 MB 37.3 MB/s 
     |████████████████████████████████| 895 kB 44.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict


torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##Configuration

In [4]:
MODEL_PATH = '/content/drive/MyDrive/NLP/IMDB/best_bert_model.pt'
DATASET_PATH = '/content/drive/MyDrive/NLP/IMDB'
PRE_TRAINED_MODEL = 'bert-base-cased'
MAX_LENGTH = 300
TOKENIZER = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL)
BATCH_SIZE = 16
# each epoch was executed separately due to google collab execution time constraints
EPOCHS = 1
DROPOUT_PROBABILITY = 0.3
N_WARMUP_STEPS = 500
LEARNING_RATE = 2e-5

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

##Class Definitions
####Dataset

In [5]:
class BERTDataset(Dataset):
  def __init__(self, dataset, labels, max_length):
    self.dataset = dataset
    self.labels = labels
    self.max_length = max_length
  
  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, item):
    text = str(self.dataset[item]) # get review text with index
    label = self.labels[item] # get label with index

    # encode review text
    encoding = TOKENIZER.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_length,
      return_token_type_ids=False,
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',
      truncation=True
    )
    return {
      'text': text,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'labels': torch.tensor(label, dtype=torch.long)
    }


####Data loader 

In [6]:
# covert pandas dataframe to Dataloader with the given batch size
def get_data_loader(df, batch_size, max_length):
  data = BERTDataset(
    dataset=df.text.to_numpy(),
    labels=df.label.to_numpy(),
    max_length=max_length
  )
  return DataLoader(data, batch_size=batch_size, num_workers=2)

####Classifier

In [7]:
class SentimentClassifier(nn.Module):
  
  def __init__(self, num_classes):
    super(SentimentClassifier, self).__init__()
    # core bert model
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL)
    # drop out layer for bert output (for regularization)
    self.drop_out = nn.Dropout(p=DROPOUT_PROBABILITY)
    # fully connected linear layer for final output 
    self.out = nn.Linear(self.bert.config.hidden_size, num_classes)
  
  def forward(self, input_ids, attention_mask):
    _, output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      return_dict=False
    )
    output = self.drop_out(output)
    output = self.out(output)
    return output

####Training

In [8]:
def train_epoch(model, data_loader, loss_fn, scheduler, optimizer,
                device, n_samples):
  
  model = model.train()
  batch_count = 1
  
  for d in data_loader:
    if batch_count % 100 == 0:
      print(f"Batch: {batch_count}/{len(data_loader)}")
    batch_count += 1
    
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["labels"].to(device)
    
    # set zerpo gradients
    optimizer.zero_grad()

    # train
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    
    # find loss
    loss = loss_fn(outputs, targets)
    loss.backward()

    # clip gradients - to prevent gradients from increasing drastically
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    # iterate optimizer
    optimizer.step()

    # iterate scheduler step
    scheduler.step()
    
    

####Evaluation

In [9]:
def eval_model(model, data_loader, loss_fn, device, n_samples):
  
  model = model.eval()
  loss_vals = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["labels"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )

      # find predictions based on the class with higher output value
      _, predictions = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)
      correct_predictions += torch.sum(predictions == targets)
      
      loss_vals.append(loss.item())
      
  return correct_predictions.double() / n_samples, np.mean(loss_vals)

####Prediction

In [10]:
def predict(model, data_loader):
  model = model.eval()
  
  predictions = []
  labels = []

  with torch.no_grad():
    for d in data_loader:

      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["labels"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      predictions.extend(preds)
      labels.extend(targets)

  return torch.stack(predictions).cpu(), torch.stack(labels).cpu()

#### Model Save/Load

In [11]:
def save_model(accuracy):
  path = f"{MODEL_PATH}" 
  torch.save({'accuracy': accuracy, 'model': model.state_dict()}, path)

def load_model():
  state = None
  try:
    state = torch.load(f'{MODEL_PATH}')
  except FileNotFoundError:
    pass
  if state:
    model.load_state_dict(state['model'])
  return state['accuracy'] if state else 0

## Load CSV Data

In [12]:
class_names = ['Negative', 'Positive']

train_data = pd.read_csv(f'{DATASET_PATH}/Train.csv')
validation_data = pd.read_csv(f'{DATASET_PATH}/Valid.csv')
test_data = pd.read_csv(f'{DATASET_PATH}/Test.csv')

train_data_loader = get_data_loader(train_data, BATCH_SIZE, MAX_LENGTH)
val_data_loader = get_data_loader(validation_data, 2*BATCH_SIZE, MAX_LENGTH)
test_data_loader = get_data_loader(test_data, 2*BATCH_SIZE, MAX_LENGTH)

##Training

In [13]:
#Initialize Classifier model
model = SentimentClassifier(len(class_names))
model = model.to(device)

#Initialize optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=N_WARMUP_STEPS,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)


best_accuracy = load_model()
print(f"Total train batches: {len(train_data_loader)}")

#Training loop
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}:')
  
  train_acc = train_epoch(model, train_data_loader, loss_fn,
                          scheduler, optimizer, device, len(train_data))
  
  val_acc, val_loss = eval_model(model, val_data_loader, loss_fn,
                                 device, len(validation_data))
  print(f'Validation loss {val_loss}, accuracy {val_acc}')
  print()
  
  if val_acc > best_accuracy:
    best_accuracy = val_acc
    save_model(best_accuracy)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total train batches: 2500
Epoch 1/1:
Batch: 100/2500
Batch: 200/2500
Batch: 300/2500
Batch: 400/2500
Batch: 500/2500
Batch: 600/2500
Batch: 700/2500
Batch: 800/2500
Batch: 900/2500
Batch: 1000/2500
Batch: 1100/2500
Batch: 1200/2500
Batch: 1300/2500
Batch: 1400/2500
Batch: 1500/2500
Batch: 1600/2500
Batch: 1700/2500
Batch: 1800/2500
Batch: 1900/2500
Batch: 2000/2500
Batch: 2100/2500
Batch: 2200/2500
Batch: 2300/2500
Batch: 2400/2500
Batch: 2500/2500
Validation loss 0.2211823220762193, accuracy 0.9198000000000001



##Results

In [14]:
print("Training:")
y_pred, y_true = predict(model, train_data_loader)
print(classification_report(y_true, y_pred, target_names=class_names), end="\n\n")

print("Validation:")
y_pred, y_true = predict(model, val_data_loader)
print(classification_report(y_true, y_pred, target_names=class_names), end="\n\n")

print("Testing:")
y_pred, y_true = predict(model, test_data_loader)
print(classification_report(y_true, y_pred, target_names=class_names))

Training:
              precision    recall  f1-score   support

    Negative       0.96      0.95      0.96     20019
    Positive       0.95      0.96      0.96     19981

    accuracy                           0.96     40000
   macro avg       0.96      0.96      0.96     40000
weighted avg       0.96      0.96      0.96     40000


Validation:
              precision    recall  f1-score   support

    Negative       0.93      0.91      0.92      2486
    Positive       0.91      0.93      0.92      2514

    accuracy                           0.92      5000
   macro avg       0.92      0.92      0.92      5000
weighted avg       0.92      0.92      0.92      5000


Testing:
              precision    recall  f1-score   support

    Negative       0.93      0.91      0.92      2495
    Positive       0.91      0.93      0.92      2505

    accuracy                           0.92      5000
   macro avg       0.92      0.92      0.92      5000
weighted avg       0.92      0.92      0.